# Neural Networks & Deep Learning - Project 1

### Dataset Used: Bank Customer Churn (Source: https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

### About the project:

The objective of this project is to build a model in Neural Networks which can classify whether a Bank customer would leave or not based on the customer information available in the dataset format

In [1]:
# Load the tensorflow package
import tensorflow as tf
print(tf.__version__)

2.0.0


In [2]:
tf.random.set_seed(10)

In [3]:
# Load necessary packages
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## 1. Read the dataset

In [4]:
df = pd.read_csv("bank.csv")

In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
df["Exited"].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

### 1. The given dataset has 13 independent variables, 1 dependent variable (Exited) and 10000 observations.
### 2. No missing values.
### 3. The target variable `Exited` has 2 values, 0 (No) and 1 (Yes) respectively. So, we need to build a binary classification model

## 2. Drop the columns which are unique for all users like IDs

The columns `RowNumber`, `CustomerId` and `Surname` are stored with unique values and identities. We can remove these columns as these have no significance in our model.

In [8]:
df = df.drop(labels=['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


### Object data type to numbers

In [10]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder

# instantiate labelencoder object
le = LabelEncoder()

In [12]:
df[["Geography", "Gender"]] = df[["Geography", "Gender"]].apply(lambda col: le.fit_transform(col))

In [13]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0


## 3. Distinguish the feature and target set

In [14]:
X = df.drop("Exited", axis=1).to_numpy()
y = df[["Exited"]].to_numpy()

In [15]:
X = X.astype('float32')
y = y.astype('float32')

In [16]:
X.shape, y.shape

((10000, 10), (10000, 1))

## 4. Divide the data set into train and test sets

In [17]:
from sklearn.model_selection import train_test_split

# Splitting data into train and test in 70:30 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=7)

In [18]:
X_train.shape, y_train.shape

((7000, 10), (7000, 1))

In [19]:
X_test.shape, y_test.shape

((3000, 10), (3000, 1))

## 5. Normalize the train and test data

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

In [21]:
# X_test has testing features set
X_test = scaler.fit_transform(X_test)

## 6. Initialize & build the model

In [22]:
# Import model packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

### Let us add a baseline model and test the accuracy and then we can optimize the model by introducing additional parameters and hidden layers

In [23]:
model = Sequential()

In [24]:
model.add(Dense(12, input_shape=(X_train.shape[1],), activation='relu'))
# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                132       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 13        
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(X_train, y_train, batch_size=16, epochs=50)

Train on 7000 samples
Epoch 1/50
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.5120 - accuracy: 0.7763
Epoch 2/50
7000/7000 [==============================] - 3s 373us/sample - loss: 0.4404 - accuracy: 0.8016
Epoch 3/50
7000/7000 [==============================] - 2s 273us/sample - loss: 0.4201 - accuracy: 0.8179
Epoch 4/50
7000/7000 [==============================] - 2s 287us/sample - loss: 0.4061 - accuracy: 0.8261
Epoch 5/50
7000/7000 [==============================] - 2s 256us/sample - loss: 0.3945 - accuracy: 0.8353
Epoch 6/50
7000/7000 [==============================] - 2s 218us/sample - loss: 0.3845 - accuracy: 0.8411
Epoch 7/50
7000/7000 [==============================] - 2s 227us/sample - loss: 0.3760 - accuracy: 0.8470
Epoch 8/50
7000/7000 [==============================] - 2s 243us/sample - loss: 0.3691 - accuracy: 0.8493
Epoch 9/50
7000/7000 [==============================] - 2s 277us/sample - loss: 0.3635 - accuracy: 0.8530
Epoch 10/50
7000/7000 [===

## 7. Optimize the model

### Let us introduce Dropout rate and add more hidden layers to optimize the model

In [27]:
model1 = Sequential()

# Input_shape is required for the first hidden layer, as it is the starting point. --> number of nodes.
# Implement dropout rate to avoid model to over learning. It drops 10% of the neurons in each epoch. 
# It helps every neuron to learn the patters
model1.add(Dense(12, input_shape=(X_train.shape[1],)))
model1.add(Dropout(0.1))
model1.add(Activation('relu'))

#As we want to implement Deep Learning, we are adding many layers
model1.add(Dense(24))
model1.add(Dropout(0.1))
model1.add(Activation('relu'))

model1.add(Dense(36))
model1.add(Dropout(0.1))
model1.add(Activation('relu'))

model1.add(Dense(24))
model1.add(Dropout(0.1))
model1.add(Activation('relu'))

model1.add(Dense(12))
model1.add(Activation('relu'))

In [28]:
# Adding the output layer
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes

model1.add(Dense(1))
model1.add(Activation('sigmoid'))

The breakdown of the inputs for compiling is as follows:

`optimizer`: adam The algorithm we want to use to find the optimal set of weights in the neural networks. Adam is a very efficeint variation of Stochastic Gradient Descent.

`loss`: binary_crossentropy This is the loss function used within adam. As our dependent (output variable) is Binary, we can use binary_crossentropy.

`metrics: [accuracy]` The accuracy metrics which will be evaluated(minimized) by the model. Used as accuracy criteria to imporve model performance.

In [29]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 12)                132       
_________________________________________________________________
dropout (Dropout)            (None, 12)                0         
_________________________________________________________________
activation (Activation)      (None, 12)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                312       
_________________________________________________________________
dropout_1 (Dropout)          (None, 24)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 24)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 36)               

In [31]:
model1.fit(X_train, y_train, batch_size=16, epochs=50)

Train on 7000 samples
Epoch 1/50
7000/7000 [==============================] - 7s 979us/sample - loss: 0.4891 - accuracy: 0.7910
Epoch 2/50
7000/7000 [==============================] - 2s 293us/sample - loss: 0.4187 - accuracy: 0.8204
Epoch 3/50
7000/7000 [==============================] - 2s 305us/sample - loss: 0.4007 - accuracy: 0.8377
Epoch 4/50
7000/7000 [==============================] - 2s 316us/sample - loss: 0.3918 - accuracy: 0.8344
Epoch 5/50
7000/7000 [==============================] - 2s 287us/sample - loss: 0.3866 - accuracy: 0.8374
Epoch 6/50
7000/7000 [==============================] - 2s 302us/sample - loss: 0.3793 - accuracy: 0.8401
Epoch 7/50
7000/7000 [==============================] - 3s 380us/sample - loss: 0.3811 - accuracy: 0.8434
Epoch 8/50
7000/7000 [==============================] - 2s 331us/sample - loss: 0.3710 - accuracy: 0.8476
Epoch 9/50
7000/7000 [==============================] - 2s 307us/sample - loss: 0.3756 - accuracy: 0.8457
Epoch 10/50
7000/7000 [=

### Adding more hidden layers and dropout rate haven't improved the accuracy and it remains in 86%

## 8. Predict the results using 0.5 as a threshold

In [32]:
y_pred = model1.predict(X_test)

# Print the first 5 predictions probabilities
print(y_pred[:5])

[[0.6797044 ]
 [0.08563331]
 [0.05012977]
 [0.75091594]
 [0.0688076 ]]


### To make the Confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form `true or false`. So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.

In [33]:
y_pred = (y_pred > 0.5).astype(int)

# Print the first 5 predictions
print(y_pred[:5])

[[1]
 [0]
 [0]
 [1]
 [0]]


## 9. Print the Accuracy score and confusion matrix

In [34]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

In [35]:
pd.DataFrame(cm, columns=['Predicted: No', 'Predicted: Yes'], index=['Actual: No', 'Actual: Yes'])

,Predicted: No,Predicted: Yes
Actual: No,2240,155
Actual: Yes,282,323


In [36]:
print(((cm[0][0]+cm[1][1])*100)/(len(y_test)), '% of testing data was classified correctly')

85.43333333333334 % of testing data was classified correctly


In [37]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.89      0.94      0.91      2395
         1.0       0.68      0.53      0.60       605

   micro avg       0.85      0.85      0.85      3000
   macro avg       0.78      0.73      0.75      3000
weighted avg       0.85      0.85      0.85      3000

